In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import kaggler
print(kaggler.__version__)

0.8.7


In [3]:
import warnings
warnings.filterwarnings("ignore")

import gc
import joblib
import lightgbm as lgb
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from kaggler.metrics import auc
from kaggler.model import AutoLGB
from kaggler.preprocessing import EmbeddingEncoder, LabelEncoder, TargetEncoder

Using TensorFlow backend.


In [4]:
plt.style.use('fivethirtyeight')
matplotlib.rcParams['figure.figsize'] = (8, 8)

In [5]:
data_dir = '../input'

trn = pd.read_csv(os.path.join(data_dir, "train.csv"))
tst = pd.read_csv(os.path.join(data_dir, "test.csv"))
sample = pd.read_csv(os.path.join(data_dir, "sample_submission.csv"))

y_trn = trn['target']

In [6]:
for col in trn.columns:
    print('{:>8s}: {:6d}'.format(col, trn[col].nunique()))

      id: 300000
   bin_0:      2
   bin_1:      2
   bin_2:      2
   bin_3:      2
   bin_4:      2
   nom_0:      3
   nom_1:      6
   nom_2:      6
   nom_3:      6
   nom_4:      4
   nom_5:    222
   nom_6:    522
   nom_7:   1220
   nom_8:   2215
   nom_9:  11981
   ord_0:      3
   ord_1:      5
   ord_2:      6
   ord_3:     15
   ord_4:     26
   ord_5:    192
     day:      7
   month:     12
  target:      2


In [7]:
features = [x for x in trn.columns if x not in ['id', 'target']]

features_to_emb = ['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9', 'ord_3', 'ord_4', 'ord_5']
n_emb = [16, 16, 20, 20, 30, 4, 8, 16]

features_not_to_emb = [x for x in features if x not in features_to_emb]

In [8]:
features_emb = []
for n, col in zip(n_emb, features_to_emb):
    features_emb += ['{}_{}'.format(col, i + 1) for i in range(n)]

In [9]:
n_fold = 5
seed = 42
cv = StratifiedKFold(n_splits=n_fold, random_state=seed)

In [10]:
le = LabelEncoder(min_obs=50)
te = TargetEncoder(smoothing=1, min_samples=50, cv=cv)

X_trn = pd.concat([le.fit_transform(trn[features]), te.fit_transform(trn[features], y_trn)], axis=1)
X_tst = pd.concat([le.transform(tst[features]), te.transform(tst[features])], axis=1)
features = ['le_{}'.format(col) for col in features] + ['te_{}'.format(col) for col in features]

# X_trn = le.fit_transform(trn[features])
# X_tst = le.transform(tst[features])

# X_trn = te.fit_transform(trn[features], y_trn)
# X_tst = te.transform(tst[features])

X_trn.columns = features
X_tst.columns = features
print(X_trn.shape, X_tst.shape)

(300000, 46) (200000, 46)


In [11]:
p = np.zeros((trn.shape[0],))
p_tst = np.zeros((tst.shape[0],))

features += features_emb
for i, (i_trn, i_val) in enumerate(cv.split(X_trn, y_trn), 1):
    y_trn_cv = y_trn[i_trn].reset_index(drop=True)

    ee = EmbeddingEncoder(cat_cols=features_to_emb, num_cols=[], n_emb=n_emb, random_state=seed)
    X_emb_trn = ee.fit_transform(trn.loc[i_trn, features_to_emb], y_trn_cv)
    X_emb_val = ee.transform(trn.loc[i_val, features_to_emb])
    X_emb_tst = ee.transform(tst[features_to_emb])
    
    X_trn_cv = pd.concat([X_trn.loc[i_trn].reset_index(drop=True), 
                          pd.DataFrame(X_emb_trn, columns=features_emb)], axis=1)
    X_val_cv = pd.concat([X_trn.loc[i_val].reset_index(drop=True), 
                          pd.DataFrame(X_emb_val, columns=features_emb)], axis=1)
    X_tst_cv = pd.concat([X_tst, pd.DataFrame(X_emb_tst, columns=features_emb)], axis=1)

#     X_trn_cv = X_trn.loc[i_trn].reset_index(drop=True)
#     X_val_cv = X_trn.loc[i_val].reset_index(drop=True)
   
    if i == 1:
        print('Feature selection and parameter tuning with CV #{}'.format(i))
        model = AutoLGB(objective='binary', metric='auc', sample_size=50000, random_state=seed)
        model.tune(X_trn_cv, y_trn_cv)
        
    model.fit(X_trn_cv, y_trn_cv)
    p[i_val] = model.predict(X_val_cv)
    print('AUC (CV #{}): {:.4f}'.format(i, auc(y_trn[i_val], p[i_val])))
    p_tst += model.predict(X_tst_cv) / n_fold
    
print('AUC CV: {:.4f}'.format(auc(y_trn, p)))

W1008 18:35:48.713423 4486972864 deprecation.py:323] From /Users/jeong/.conda/envs/py36/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1008 18:35:50.913494 4486972864 deprecation_wrapper.py:119] From /Users/jeong/.conda/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Train on 191999 samples, validate on 48000 samples
Epoch 1/100
191999/191999 [==============================] - 4s 19us/step - loss: 0.5981 - auc: 0.6399 - val_loss: 0.5826 - val_auc: 0.7033
Epoch 2/100
191999/191999 [==============================] - 2s 12us/step - loss: 0.5557 - auc: 0.7124 - val_loss: 0.5566 - val_auc: 0.7133
Epoch 3/100
191999/191999 [==============================] - 2s 13us/step - loss: 0.5389 - auc: 0.7372 - val_loss: 0.5638 - val_auc: 0.7072
Epoch 4/100
191999/191999 [==============================] - 2s 13us/step - loss: 0.5269 - auc: 0.7528 - val_loss: 0.5750 - val_auc: 0.7035
Epoch 5/100
191999/191999 [==============================] - 2s 13us/step - loss: 0.5156 - auc: 0.7657 - val_loss: 0.5828 - val_auc: 0.6974
Epoch 6/100
191999/191999 [==============================] - 2s 12us/step - loss: 0.4902 - auc: 0.7949 - val_loss: 0.6070 - val_auc: 0.6836
Epoch 7/100
191999/191999 [==============================] - 3s 13us/step - loss: 0.4771 - auc: 0.8088 - val_

In [12]:
print('{} features selected out of {}'.format(len(model.features), len(features)))

138 features selected out of 176


In [13]:
print("Saving submission file")
submission = pd.DataFrame.from_dict({
    'id': tst.id.values,
    'target': p_tst
})
submission.to_csv("submission.csv", index=False)

Saving submission file
